In [1]:
import pandas as pd
from sklearn import preprocessing

In [2]:
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)

In [3]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('Sources/test_final_100k.csv')
visitas = pd.read_csv('Sources/visitas_proc.csv')
preds = pd.DataFrame()

In [4]:
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(visitas, on=['idaviso','idpostulante'], how='left')
resultados = postulaciones['sepostulo']
postulaciones.drop(['titulo','descripcion','idaviso','idpostulante','sepostulo'],axis=1, inplace=True)
postulaciones.head()

,Unnamed: 0.1,fechanacimiento,sexo,nivel,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,...,prom_edad,sabados,domingos,ingles,3 anos,2 anos,4 anos,5 anos,Unnamed: 0,visito
0,177374,29.0,FEM,5,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,...,26.095588,0,0,1,0,0,0,0,NaN,NaN
1,447085,20.0,FEM,11,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,...,26.095588,0,0,1,0,0,0,0,NaN,NaN
2,145390,38.0,MASC,3,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,...,26.095588,0,0,1,0,0,0,0,NaN,NaN
3,460866,NaN,NO_DECLARA,-1,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,...,26.095588,0,0,1,0,0,0,0,NaN,NaN
4,358106,22.0,FEM,5,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,...,26.095588,0,0,1,0,0,0,0,NaN,NaN


In [5]:
postulaciones.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1,inplace=True)

In [6]:
visitas.head()

,Unnamed: 0,idpostulante,idaviso,visito
0,0,0002q,1789742,4
1,1,0002q,1807692,6
2,2,0002q,1808090,5
3,3,0005E,1744005,3
4,4,0005E,1785430,2


In [7]:
test = test.merge(postulantes, on='idpostulante')
test = test.merge(avisos, on='idaviso')
test = test.merge(visitas, on=['idaviso','idpostulante'], how='left')
test.drop(['titulo','descripcion','idaviso','idpostulante','id','Unnamed: 0.1','Unnamed: 0'],axis=1, inplace=True)
test.head()

,fechanacimiento,sexo,nivel,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,...,TendenciaSexo,prom_edad,sabados,domingos,ingles,3 anos,2 anos,4 anos,5 anos,visito
0,42.0,FEM,21,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,NaN,NaN,...,NaN,NaN,0,0,1,0,0,0,0,NaN
1,31.0,MASC,9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,NaN,NaN,...,NaN,NaN,0,0,1,0,0,0,0,NaN
2,36.0,FEM,12,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,NaN,NaN,...,NaN,NaN,0,0,1,0,0,0,0,NaN
3,69.0,MASC,15,Full-time,Otro,Tecnologia / Sistemas,TeleTech,214.0,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
4,32.0,FEM,11,Full-time,Otro,Tecnologia / Sistemas,TeleTech,214.0,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN


In [8]:
import numpy as np

In [9]:
test = pd.concat([test,postulaciones])

In [10]:
test.columns

Index(['fechanacimiento', 'sexo', 'nivel', 'tipo_de_trabajo', 'nivel_laboral',
       'nombre_area', 'denominacion_empresa', 'cluster', 'FEM', 'MASC',
       'NO_DECLARA', 'TendenciaSexo', 'prom_edad', 'sabados', 'domingos',
       'ingles', '3 anos', '2 anos', '4 anos', '5 anos', 'visito'],
      dtype='object')

In [11]:
test['fechanacimiento'].fillna(np.nanmean(test['fechanacimiento']),inplace=True)

In [12]:
test['FEM'].fillna(np.nanmean(test['FEM']),inplace=True)
test['MASC'].fillna(np.nanmean(test['MASC']),inplace=True)
test['NO_DECLARA'].fillna(np.nanmean(test['NO_DECLARA']),inplace=True)
test['prom_edad'].fillna(np.nanmean(test['prom_edad']),inplace=True)
test['FEM'].fillna(np.nanmean(test['FEM']),inplace=True)
lista = ['sabados', 'domingos','ingles', '3 anos', '2 anos', '4 anos', '5 anos', 'visito']
for i in lista:
    test[i].fillna(0,inplace=True)

In [13]:
test.head()

,fechanacimiento,sexo,nivel,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,...,TendenciaSexo,prom_edad,sabados,domingos,ingles,3 anos,2 anos,4 anos,5 anos,visito
0,42.0,FEM,21,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,673.024622,437.282982,...,NaN,30.794663,0,0,1,0,0,0,0,0.0
1,31.0,MASC,9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,673.024622,437.282982,...,NaN,30.794663,0,0,1,0,0,0,0,0.0
2,36.0,FEM,12,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,45.0,673.024622,437.282982,...,NaN,30.794663,0,0,1,0,0,0,0,0.0
3,69.0,MASC,15,Full-time,Otro,Tecnologia / Sistemas,TeleTech,214.0,673.024622,437.282982,...,NaN,30.794663,0,0,0,0,0,0,0,0.0
4,32.0,FEM,11,Full-time,Otro,Tecnologia / Sistemas,TeleTech,214.0,673.024622,437.282982,...,NaN,30.794663,0,0,0,0,0,0,0,0.0


In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not postulaciones[col].dtype == 'O': continue
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])

In [15]:
postulaciones = test[100000:].fillna(-1)
test = test[:100000].fillna(-1)

In [16]:
partitions = []
partitions_res = []
n = 5
index = int(postulaciones.shape[0]/n)
for i in range(n):
    partitions.append(postulaciones[index*i:index*(i+1)])
    partitions_res.append(resultados[index*i:index*(i+1)])


In [17]:
aviso = None
postulantes = None
visitas = None

In [18]:
def proba(result):
    prob_1 = []
    for x in result:
        prob_1.append(x[1])
    return prob_1

In [19]:
print('Arranca XG')

import xgboost as xgb
dtrain = xgb.DMatrix(data=partitions[0],label=partitions_res[0])
dtest = xgb.DMatrix(data=partitions[4])
param = {'max_depth':20, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'subsample':0.8}
num_round = 50
bst = xgb.train(param, dtrain, num_round)
preds['xgb'] = bst.predict(dtest)

print('Termino XG')

Arranca XG
Termino XG


In [20]:
print('Arranca RF')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
clf=RandomForestClassifier(n_estimators=50, warm_start=True, criterion='entropy', n_jobs=-1)
clf.fit(partitions[1],partitions_res[1])
preds['RF'] = proba(clf.predict_proba(partitions[4]))

print('Termino RF')

Arranca RF
Termino RF


In [21]:
print('Arranca NB')

from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(partitions[2] ,partitions_res[2])
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
preds['NB'] = proba(nb.predict_proba(partitions[4]))

print('Termino NB')

Arranca NB
Termino NB


In [23]:
print('Arranca PERCEPTRON')
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(partitions[3], partitions_res[3])
preds['PERCEPTRON'] = proba(clf.predict_proba(partitions[4]))
#from sklearn import svm
#clf = svm.SVC()
#clf.fit(preds, partitions_res[4])

print('PERCEPTRON ya esta entrenado')

Arranca PERCEPTRON
PERCEPTRON ya esta entrenado


In [24]:
preds.to_csv('Sources/predicciones.csv')

In [25]:
print('Arranca KNN')

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=25 ,n_jobs=-1, algorithm='auto')
neigh.fit(preds, partitions_res[4])

print('Termino KNN')



Arranca KNN
Termino KNN


In [26]:
preds.head()

,xgb,RF,NB,PERCEPTRON
0,0.999554,0.980000,0.997866,0.225699
1,0.893244,0.817318,0.411786,0.225699
2,0.801883,0.729619,0.411786,0.225699
3,0.694682,0.764000,0.411786,0.225699
4,0.074018,0.598794,0.411786,0.225699


In [27]:
final = pd.DataFrame()
test_final = pd.read_csv('Sources/test_final_100k.csv')['id']

In [28]:
print('Arranca XGB final')

aviso = None
postulantes = None
import xgboost as xgb
dtrain = xgb.DMatrix(data=postulaciones, label=resultados)
dtest = xgb.DMatrix(data=test)
param = {'max_depth':20, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'subsample':0.8}
num_round = 50
bst = xgb.train(param, dtrain, num_round)
final['xgb'] = bst.predict(dtest)

print('Termino XGB final')

Arranca XGB final
Termino XGB final


In [29]:
test_final['sepostulo'] = final['xgb']
test_final.to_csv('Sources/1.csv',index=False)

In [30]:
print('Arranca RF final')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
clfr=RandomForestClassifier(n_estimators=50, warm_start=True, criterion='entropy', n_jobs=-1)
clfr.fit(postulaciones,resultados)
final['RF'] = proba(clfr.predict_proba(test))

print('Termino RF final')

Arranca RF final
Termino RF final


In [31]:
test_final['sepostulo'] = final['RF']
test_final.to_csv('Sources/2.csv',index=False)

In [32]:
print('Arranca NB final')

from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(postulaciones, resultados)
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
final['NB'] = proba(nb.predict_proba(test))

print('Termino NB final')

Arranca NB final
Termino NB final


In [33]:
test_final['sepostulo'] = final['NB']
test_final.to_csv('Sources/3.csv',index=False)

In [35]:
print('Arranca PPERCEPTRON final')
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(postulaciones, resultados)
final['PERCEPTRON'] = proba(clf.predict_proba(test))

print('Termino PERCEPTRON final')

Arranca PPERCEPTRON final
Termino PERCEPTRON final


In [36]:
test_final['sepostulo'] = final['PERCEPTRON']
test_final.to_csv('Sources/4.csv',index=False)

In [37]:
final.head()

,xgb,RF,NB,PERCEPTRON
0,4.927957e-08,0.0,0.205207,0.330076
1,3.521454e-06,0.0,0.205207,0.330076
2,1.623599e-06,0.0,0.205207,0.330076
3,4.359569e-06,0.0,0.251252,0.330076
4,4.836552e-08,0.0,0.251252,0.330076


In [38]:
print('Predigo con KNN')

resultado = proba(neigh.predict(final))

print('termino KNN')

Predigo con KNN
termino KNN


In [39]:
resultado

[0.0,
 0.0,
 0.0,
 0.08,
 0.08,
 0.08,
 0.0,
 0.0,
 0.0,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.04,
 0.04,
 0.04,
 0.0,
 0.08,
 0.08,
 0.0,
 0.0,
 0.0,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.08,
 0.0,
 0.0,
 0.0,
 0.0,
 0.08,
 0.08,
 0.08,
 0.0,
 0.0,
 0.0,
 1.0,
 0.92,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8,
 1.0,
 0.28,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8,
 1.0,
 1.0,
 0.92,
 1.0,
 1.0,
 1.0,
 1.0,
 0.84,
 1.0,
 0.68,
 0.32,
 1.0,
 0.8,
 0.84,
 0.4,
 1.0,
 1.0,
 0.76,
 0.16,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.16,
 0.68,
 1.0,
 0.96,
 1.0,
 1.0,
 0.8,
 0.92,
 0.84,
 1.0,
 1.0,
 1.0,
 1.0,
 0.64,
 0.72,
 1.0,
 0.72,
 1.0,
 0.88,
 0.88,
 0.32,
 1.0,
 1.0,
 1.0,
 0.76,
 0.16,
 1.0,
 1.0,
 1.0,
 0.68,
 1.0,
 0.96,
 1.0,
 0.12,
 1.0,
 0.96,
 1.0,
 1.0,
 0.96,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.96,
 1.0,
 1.0,
 1.0,
 1.0,
 0.96,
 1

In [40]:
test = pd.read_csv('Sources/test_final_100k.csv')
test['sepostulo'] = resultado
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultado_final_pipe.csv', index=False)

In [44]:
resultado[resultado > 0.5 ].shape

TypeError: '>' not supported between instances of 'list' and 'float'